In [1]:
from llama_cpp import Llama
from rich import print
from langchain_core.utils.function_calling import convert_to_openai_function
from llama_cpp.llama_tokenizer import LlamaHFTokenizer

In [2]:
llm = Llama.from_pretrained(
    repo_id="meetkai/functionary-small-v2.2-GGUF",
    filename="functionary-small-v2.2.q4_0.gguf",
    chat_format="functionary-v2",
    tokenizer=LlamaHFTokenizer.from_pretrained("meetkai/functionary-small-v2.2-GGUF"),
    n_gpu_layers=-1,
    verbose=False,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from langchain.tools import tool
import requests
import json


@tool
def get_current_weather(city: str) -> str:
    """
    Get the current weather for a given city.

    Args:
      city (str): The city to fetch weather for.

    Returns:
      str: current weather condition, or None if an error occurs.
    """
    try:
        data = (
            requests.get(f"https://wttr.in/{city}?format=j1")
            .json()
            .get("current_condition")
        )
        data = json.dumps(data, separators=(",", ":"))
        return data
    except Exception as e:
        print(f"Error fetching current weather for {city}: {e}")
        return None

In [4]:
oai_tool = convert_to_openai_function(get_current_weather)

In [5]:
oai_tool

{'name': 'get_current_weather',
 'description': 'get_current_weather(city: str) -> str - Get the current weather for a given city.\n\nArgs:\n  city (str): The city to fetch weather for.\n\nReturns:\n  str: current weather condition, or None if an error occurs.',
 'parameters': {'type': 'object',
  'properties': {'city': {'type': 'string'}},
  'required': ['city']}}

In [6]:
print(oai_tool)

{
    'name': 'get_current_weather',
    'description': 'get_current_weather(city: str) -> str - Get the current weather for a given city.\n\nArgs:\n  
city (str): The city to fetch weather for.\n\nReturns:\n  str: current weather condition, or None if an error 
occurs.',
    'parameters': {'type': 'object', 'properties': {'city': {'type': 'string'}}, 'required': ['city']}
}

In [7]:
output = llm.create_chat_completion(
    messages=[{"role": "user", "content": "How is the weather in London today?"}],
    tools=[{"type": "function", "function": oai_tool}],
)

In [8]:
print(output)

{
    'id': 'chatcmpl-783e2310-e0b5-415b-af2f-792c9d80e8c7',
    'object': 'chat.completion',
    'created': 1715016535,
    'model': 
'/Users/aniket/.cache/huggingface/hub/models--meetkai--functionary-small-v2.2-GGUF/snapshots/9971ff5d1eacbfac81fe69
39bccd59cf7d668078/./functionary-small-v2.2.q4_0.gguf',
    'choices': [
        {
            'index': 0,
            'logprobs': None,
            'message': {
                'role': 'assistant',
                'content': None,
                'tool_calls': [
                    {
                        'id': 'call_IozW3gLYtfatBZsDhhfmzU6D',
                        'type': 'function',
                        'function': {'name': 'get_current_weather', 'arguments': '{"city": "London"}'}
                    }
                ]
            },
            'finish_reason': 'tool_calls'
        }
    ],
    'usage': {'prompt_tokens': 198, 'completion_tokens': 16, 'total_tokens': 199}
}